In [98]:
import sys
from sqlalchemy import create_engine
import pymysql
import pandas as pd

In [ ]:
import random
random.seed(1030)

In [ ]:
import numpy as np
import datetime

### Paths and Connection Details

In [175]:
exec(open("/Users/walkerag/Documents/osu/cs467/project_paths.py").read())

In [132]:
db_connection = create_engine(db_connection_str)

### Tables

In [133]:
all_tables = pd.read_sql('SHOW TABLES', con=db_connection)
print(all_tables)

# Describe all the DB tables
def describe_table(table, db_connection):
        
    print(table)    
    table_cols = pd.read_sql('DESCRIBE ' + table, con=db_connection)
    print(table_cols)

result = [describe_table(table, db_connection) for table in all_tables['Tables_in_project']]

           Tables_in_project
0                    Country
1                       Item
2             Price_Feedback
3             Price_Response
4              Shopping_List
5      Shopping_List_History
6                      State
7                      Store
8             Store_Feedback
9    Store_Feedback_Category
10            Store_Response
11                      User
12           User_Reputation
13  User_Reputation_Category
Country
          Field              Type Null  Key Default           Extra
0    country_id  int(10) unsigned   NO  PRI    None  auto_increment
1  country_name       varchar(50)   NO         None                
Item
              Field              Type Null  Key Default           Extra
0           item_id  int(10) unsigned   NO  PRI    None  auto_increment
1         item_name       varchar(50)   NO         None                
2          item_upc       varchar(12)   NO         None                
3  item_description      varchar(200)   NO         None     

### Load Data

In [114]:
# Function to load data from a CSV into MySQL database
def data_loader_csv(data_path, table_name, db_connection, load_type):
    
    print("Loading data from CSV")
    
    # Read in from CSV
    df = pd.read_csv(data_path + 'Input Data - ' + table_name.lower() + '.csv')
    
    print(df.head(5))
    
    # Load to mySQL
    df.to_sql(table_name, con = db_connection, if_exists = load_type, index = False)
    
    # Check table
    results_df = pd.read_sql('SELECT * FROM ' + table_name + ' LIMIT 5', con=db_connection)
    print(results_df)

In [126]:
data_loader_csv(data_path, "Country", db_connection, "append")

Loading data from CSV
     country_name
0     Afghanistan
1         Albania
2         Algeria
3  American Samoa
4         Andorra
   country_id    country_name
0           8     Afghanistan
1           9         Albania
2          10         Algeria
3          11  American Samoa
4          12         Andorra


In [115]:
data_loader_csv(data_path, "State", db_connection, "append")

Loading data from CSV
  state_name
0         AL
1         AK
2         AZ
3         AR
4         CA
   state_id state_name
0         1         AL
1         2         AK
2         3         AZ
3         4         AR
4         5         CA


In [ ]:
data_loader_csv(data_path, "Item", db_connection, "append")

In [128]:
data_loader_csv(data_path, "User", db_connection, "append")

Loading data from CSV
  first_name last_name                              email signup_date  \
0       Adam    Walker       adamwalker@notarealemail.com  2020-04-20   
1      James      Dean        jamesdean@notarealemail.com  2020-04-20   
2    Zackary     Morse     zackarymorse@notarealemail.com  2020-04-20   
3  Heriberto    Mellor  heribertomellor@notarealemail.com  2020-04-20   
4      Elton    Bussey      eltonbussey@notarealemail.com  2020-04-20   

   last_login  user_country  
0  2020-04-20           235  
1  2020-04-20           235  
2  2020-04-20           235  
3  2020-04-20           235  
4  2020-04-20           235  
   user_id first_name last_name                              email  \
0        1       Adam    Walker       adamwalker@notarealemail.com   
1        2      James      Dean        jamesdean@notarealemail.com   
2        3    Zackary     Morse     zackarymorse@notarealemail.com   
3        4  Heriberto    Mellor  heribertomellor@notarealemail.com   
4        

In [139]:
data_loader_csv(data_path, "Store", db_connection, "append")

Loading data from CSV
     store_name  store_lat  store_long  store_active_flag  \
0  Trader Joe's          0           0                  1   
1  Trader Joe's          0           0                  1   
2  Trader Joe's          0           0                  1   
3  Trader Joe's          0           0                  1   
4  Trader Joe's          0           0                  1   

          store_street           store_city  store_state  store_zip  
0  417 Westlake Center            Daly City            5      94015  
1   301 McLellan Drive  South San Francisco            5      94080  
2       265 Winston Dr        San Francisco            5      94132  
3           401 Bay St        San Francisco            5      94133  
4        3 Masonic Ave        San Francisco            5      94118  
   store_id    store_name  store_lat  store_long  store_active_flag  \
0        67  Trader Joe's        0.0         0.0                  1   
1        68  Trader Joe's        0.0         0.0 

In [3]:
cnx = mysql.connector.connect(user='root', password='cs467project',
                              host='localhost',
                              database='romanesco',
                              use_pure=False)

In [ ]:
df = pd.read_sql('DELETE FROM Store', con=db_connection)
print(df)

In [48]:
df = pd.read_sql('DESCRIBE Country', con=db_connection)
print(df)

          Field              Type Null  Key Default           Extra
0    country_id  int(10) unsigned   NO  PRI    None  auto_increment
1  country_name       varchar(50)   NO         None                


In [ ]:
df = pd.read_sql('SELECT * FROM Country', con=db_connection)
print(df)

### Sklearn

In [11]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier

In [12]:
X, y = make_hastie_10_2(random_state=0)

In [15]:
X[1:10,1:10]

array([[ 1.45427351,  0.76103773,  0.12167502,  0.44386323,  0.33367433,
         1.49407907, -0.20515826,  0.3130677 , -0.85409574],
       [ 0.6536186 ,  0.8644362 , -0.74216502,  2.26975462, -1.45436567,
         0.04575852, -0.18718385,  1.53277921,  1.46935877],
       [ 0.37816252, -0.88778575, -1.98079647, -0.34791215,  0.15634897,
         1.23029068,  1.20237985, -0.38732682, -0.30230275],
       [-1.42001794, -1.70627019,  1.9507754 , -0.50965218, -0.4380743 ,
        -1.25279536,  0.77749036, -1.61389785, -0.21274028],
       [ 0.3869025 , -0.51080514, -1.18063218, -0.02818223,  0.42833187,
         0.06651722,  0.3024719 , -0.63432209, -0.36274117],
       [-0.35955316, -0.81314628, -1.7262826 ,  0.17742614, -0.40178094,
        -1.63019835,  0.46278226, -0.90729836,  0.0519454 ],
       [ 0.12898291,  1.13940068, -1.23482582,  0.40234164, -0.68481009,
        -0.87079715, -0.57884966, -0.31155253,  0.05616534],
       [ 0.90082649,  0.46566244, -1.53624369,  1.48825219,  1

In [16]:
X_train, X_test = X[:2000], X[2000:]

In [17]:
y_train, y_test = y[:2000], y[2000:]

In [18]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(X_train, y_train)

In [19]:
clf.score(X_test, y_test)

0.913

### Data Simulation

In [140]:
# Function to create and upload a shopping list for a given user_id
# min_items and max_items limit number of items in list
def _fill_list(user_id, min_items, max_items, items):
    
    # Determine total items in list
    tot_items = np.random.uniform(min_items, max_items, 1).round().astype(int)
    print(tot_items)

    # Get item IDs
    list_items = np.random.choice(items['item_id'], tot_items, replace=False)

    # Convert to pandas dataframe
    dataset = pd.DataFrame({'item_id': list_items})

    # Format dataframe
    dataset['user_id'] = user_id
    dataset['time_added'] = datetime.datetime.now()
    dataset['time_removed'] = None
    dataset = dataset[['user_id', 'item_id','time_added','time_removed']]

    # Load to mySQL
    print("Loading")
    dataset.to_sql('Shopping_List_History', con = db_connection, if_exists = "append", index = False)

# Generate shopping list by sampling from items table
def generate_shopping_list_history(db_connection, min_items, max_items):
    
    # Get items
    items = pd.read_sql('SELECT DISTINCT item_id FROM Item', con=db_connection)
    
    # Users
    users = pd.read_sql('SELECT DISTINCT user_id FROM User', con=db_connection)
    
    # Loop over users
    result = [_fill_list(user_id, min_items, max_items, items) for user_id in users['user_id']]

generate_shopping_list_history(db_connection, 1, 40)

In [ ]:
# Check result
df = pd.read_sql('SELECT * FROM Shopping_List_History LIMIT 100', con=db_connection)
print(df)

In [141]:
# Get distinct chains
store_names = pd.read_sql('SELECT DISTINCT store_name FROM Store', con=db_connection)
print(store_names)

#
stores_names['store_quality'] = 0

           store_name
0        Trader Joe's
1             Safeway
2  Whole Foods Market
3              Costco


In [145]:
s = np.random.uniform(-1,0,10)

In [146]:
s

array([-0.80597878, -0.89860904, -0.46956359, -0.09330859, -0.06566857,
       -0.26490908, -0.66790888, -0.56057857, -0.53782422, -0.1592114 ])

In [ ]:
# Get users




In [148]:
items = pd.read_sql('SELECT DISTINCT item_id FROM Item', con=db_connection)

In [149]:
users = pd.read_sql('SELECT DISTINCT user_id FROM User', con=db_connection)

In [ ]:
def describe_table(table, db_connection):
        
    print(table)    
    table_cols = pd.read_sql('DESCRIBE ' + table, con=db_connection)
    print(table_cols)

result = [fill_list(user_id, min_items, max_items) for user_id in users['user_id']]

In [170]:
df = pd.read_sql('SELECT * FROM Shopping_List_History', con=db_connection)
print(df)

   shopping_list_history_id  user_id  item_id          time_added time_removed
0                         1        3       19 2020-04-22 06:18:58         None
1                         2        3       11 2020-04-22 06:18:58         None
2                         3        3       50 2020-04-22 06:18:58         None
3                         4        3       17 2020-04-22 06:18:58         None
4                         5        3       34 2020-04-22 06:18:58         None
5                         6        3       48 2020-04-22 06:18:58         None


In [ ]:
df = pd.read_sql('DELETE FROM Shopping_List_History', con=db_connection)
print(df)

[15]
Loading
[30]
Loading
[39]
Loading
[12]
Loading
[35]
Loading
[30]
Loading
[28]
Loading
[33]
Loading
[8]
Loading
[6]
Loading
[1]
Loading
[32]
Loading
[26]
Loading
[10]
Loading
[2]
Loading
[25]
Loading
[20]
Loading
[39]
Loading
[7]
Loading
[17]
Loading
[17]
Loading
[13]
Loading
[31]
Loading
[13]
Loading
[10]
Loading
[12]
Loading
[25]
Loading
[24]
Loading
[32]
Loading
[7]
Loading
[25]
Loading
[5]
Loading
[16]
Loading
[36]
Loading
[18]
Loading
[34]
Loading
[14]
Loading
[33]
Loading
[15]
Loading
[3]
Loading
[17]
Loading
[13]
Loading
[15]
Loading
[16]
Loading
[31]
Loading
[6]
Loading
[3]
Loading
[26]
Loading
[36]
Loading
[23]
Loading
[28]
Loading
[20]
Loading
[20]
Loading
[33]
Loading
[16]
Loading
[25]
Loading
[12]
Loading
[8]
Loading
[32]
Loading
[40]
Loading
[22]
Loading
[30]
Loading
[38]
Loading
[3]
Loading
[34]
Loading
[24]
Loading
[27]
Loading
[10]
Loading
[37]
Loading
[39]
Loading
[39]
Loading
[29]
Loading
[13]
Loading
[34]
Loading
[35]
Loading
[21]
Loading
[9]
Loading
[9]
Loading


### Scrap

In [ ]:
# initialize list of lists 
data = [['United States of America'], ['Brazil'], ['Spain']] 
  
# Create the pandas DataFrame 
df = pd.DataFrame(data, columns = ['country_name']) 
  
# print dataframe. 
df 